In [1]:
%pip install snowflake-snowpark-python
%pip install boto3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc
import snowflake.snowpark
from snowflake.snowpark.types import IntegerType
import boto3

In [3]:
import logging
import sys

In [4]:
with open("./secrets/config.json", "r") as f:
    connection_parameters = json.loads(f.read())
test_session = Session.builder.configs(connection_parameters).create()

In [5]:
with open("./secrets/greenhouse.json", "r") as f:
    greenhouse_apikey = json.loads(f.read())["api_key"]

In [6]:
bucket = "airbyte.alex"
connector = "source-pokeapi"
s3_paginator = boto3.client('s3').get_paginator('list_objects_v2')

def keys(bucket_name, prefix='/', delimiter='/', start_after=''):
    substrings_to_ignore = ('.dockerignore', '.DS_Store', '__', '.venv')
    prefix = prefix[1:] if prefix.startswith(delimiter) else prefix
    start_after = (start_after or prefix) if prefix.endswith(delimiter) else start_after
    for page in s3_paginator.paginate(Bucket=bucket_name, Prefix=prefix, StartAfter=start_after):
        for content in page.get('Contents', ()):
            key = content['Key']
            ignore = any([s in key for s in substrings_to_ignore])
            if ignore:
                continue
            else:
                yield key
            
keys = list(keys(bucket, prefix=f"{connector}"))

# Compute the list of files to import in the proc
# For now we're just loading everything
files_to_load = [f"@mystage/{k}" for k in keys if "pendulum" not in k]
files_to_load

['@mystage/source-pokeapi/airbyte_cdk.zip',
 '@mystage/source-pokeapi/attr.zip',
 '@mystage/source-pokeapi/attrs.zip',
 '@mystage/source-pokeapi/backoff.zip',
 '@mystage/source-pokeapi/bin.zip',
 '@mystage/source-pokeapi/certifi.zip',
 '@mystage/source-pokeapi/charset_normalizer.zip',
 '@mystage/source-pokeapi/coverage.zip',
 '@mystage/source-pokeapi/dataclasses_jsonschema.zip',
 '@mystage/source-pokeapi/dateutil.zip',
 '@mystage/source-pokeapi/deepdiff.zip',
 '@mystage/source-pokeapi/deprecated.zip',
 '@mystage/source-pokeapi/distutils-precedence.pth',
 '@mystage/source-pokeapi/docker.zip',
 '@mystage/source-pokeapi/dpath.zip',
 '@mystage/source-pokeapi/exceptiongroup.zip',
 '@mystage/source-pokeapi/fancycompleter.py',
 '@mystage/source-pokeapi/hypothesis.zip',
 '@mystage/source-pokeapi/hypothesis_jsonschema.zip',
 '@mystage/source-pokeapi/icdiff.py',
 '@mystage/source-pokeapi/idna.zip',
 '@mystage/source-pokeapi/inflection.py',
 '@mystage/source-pokeapi/inflection.zip',
 '@mystage/so

In [17]:
import snowflake.snowpark
from snowflake.snowpark.functions import sproc
from snowflake.snowpark.types import IntegerType

# Create a stage to store the code
create_stage_result = test_session.sql(f"create or REPLACE stage mystage url = 's3://{bucket}'").collect()

# pendulum has to be installed as a package for reasons unknown...
@sproc(packages=['snowflake-snowpark-python', 'pendulum'], imports=files_to_load, name="myproc2", replace=True)
def compute(session: snowflake.snowpark.Session, to_table: str, config: dict) -> str:
    from airbyte_cdk.models import ConfiguredAirbyteCatalog, ConfiguredAirbyteStream
    from airbyte_cdk.models import SyncMode, DestinationSyncMode

#    from source_faker import SourceFaker
    from source_pokeapi import SourcePokeapi
    logger = logging.getLogger("logger")
#    source = SourceFaker()
    source = SourcePokeapi()
#    config = {"count": 5, "seed": -1, "records_per_sync": 10, "records_per_slice": 3}
    #TODO: need to patch the read method
    catalog = source.discover(logger, config)
    configured_catalog = ConfiguredAirbyteCatalog(
        streams=[ConfiguredAirbyteStream(stream=s, sync_mode=SyncMode.full_refresh, destination_sync_mode=DestinationSyncMode.append) for s
                 in catalog.streams])
    #data = [1, 2, 3, 4]
    data = source.read(logger, config, configured_catalog, {})
    session.create_dataframe(data).write.mode('append').save_as_table(to_table)
    return "SUCCESS"

In [ ]:
#print(test_session.sql("select count(*) from public.test_table3;").collect())
print(test_session.call("myproc2", "public.test_table3", {"pokemon_name": "ditto"}))
#print(test_session.sql("select * from public.test_table3;").collect())

In [ ]:
#test_session.close()

In [49]:
# Create the API Integration object
# https://docs.snowflake.com/en/sql-reference/external-functions-creating-aws-common-api-integration.html#create-the-api-integration-object
output_from_external_function = test_session.sql("select ALEX_TEST_DB.PUBLIC.test_external_function();").collect()
records = json.loads(output_from_external_function[0].as_dict()['ALEX_TEST_DB.PUBLIC.TEST_EXTERNAL_FUNCTION()'])["list"]

In [53]:
test_session.sql(f"create secret service_now_creds_pw type = password    username = '{greenhouse_apikey}'    password = '';").collect()

Failed to execute query [queryID: 01a6ee22-0000-be18-0002-c0a6000110fe] create secret service_now_creds_pw type = password    username = '9e35eb3012c2b87a16e55a8c2a9d1a03-3'    password = '';
000002 (0A000): Unsupported feature 'TOK_SECRET'.


SnowparkSQLException: (1304): 000002 (0A000): Unsupported feature 'TOK_SECRET'.

In [55]:
test_session.sql('select current_account();').collect()

[Row(CURRENT_ACCOUNT()='YP50190')]